In [134]:
import os 
import folium
from folium import plugins
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy as et
from geopy.geocoders import Nominatim
import csv
import pandas as pd
import math

In [105]:
trees = pd.read_csv('trees.csv')
trees

,name,sci_name,state,county,points,link,img,lat,long
0,Sequoia Giant,Sequoiadendron giganteum,CA,Tulare,1321,https://www.americanforests.org?page_id=56124,https://www.americanforests.org/wp-content/plu...,36.251647,-118.852583
1,Redwood Coast,Sequoia sempervirens,CA,Del Norte,1223,https://www.americanforests.org?page_id=85016,https://www.americanforests.org/wp-content/plu...,41.726177,-123.913280
2,Redwood Coast,Sequoia sempervirens,CA,Humboldt,1216,https://www.americanforests.org?page_id=84992,https://www.americanforests.org/wp-content/plu...,40.599742,-123.899773
3,Spruce Sitka,Picea sitchensis,WA,Grays Harbor,951,https://www.americanforests.org?page_id=56046,https://www.americanforests.org/wp-content/plu...,47.174417,-123.865883
4,Redcedar Western,Thuja plicata,WA,Clallam,922,https://www.americanforests.org?page_id=64651,https://www.americanforests.org/wp-content/plu...,47.999838,-123.737347
...,...,...,...,...,...,...,...,...,...
639,Snowbell Bigleaf,Styrax grandifolius,VA,Chesapeake,31,https://www.americanforests.org?page_id=84999,https://www.americanforests.org/wp-content/plu...,36.718371,-76.246680
640,Anise-tree Yellow,Illicium parviflorum,FL,Marion,30,https://www.americanforests.org?page_id=84975,https://www.americanforests.org/wp-content/plu...,29.218245,-82.062695
641,Hawthorn Reverchon,Crataegus reverchonii,TX,Dallas,25,https://www.americanforests.org?page_id=55561,https://www.americanforests.org/wp-content/plu...,32.762040,-96.779007
642,Bayberry Evergreen,Morella caroliniensis,VA,Newport News,24,https://www.americanforests.org?page_id=56398,https://www.americanforests.org/wp-content/plu...,37.091443,-76.521389


In [135]:
trees['qtile'] = pd.qcut(trees['points'], 4, labels = False) + 1

,name,sci_name,state,county,points,link,img,lat,long,pctile,qtile
0,Sequoia Giant,Sequoiadendron giganteum,CA,Tulare,1321,https://www.americanforests.org?page_id=56124,https://www.americanforests.org/wp-content/plu...,36.251647,-118.852583,4,4
1,Redwood Coast,Sequoia sempervirens,CA,Del Norte,1223,https://www.americanforests.org?page_id=85016,https://www.americanforests.org/wp-content/plu...,41.726177,-123.913280,4,4
2,Redwood Coast,Sequoia sempervirens,CA,Humboldt,1216,https://www.americanforests.org?page_id=84992,https://www.americanforests.org/wp-content/plu...,40.599742,-123.899773,4,4
3,Spruce Sitka,Picea sitchensis,WA,Grays Harbor,951,https://www.americanforests.org?page_id=56046,https://www.americanforests.org/wp-content/plu...,47.174417,-123.865883,4,4
4,Redcedar Western,Thuja plicata,WA,Clallam,922,https://www.americanforests.org?page_id=64651,https://www.americanforests.org/wp-content/plu...,47.999838,-123.737347,4,4
...,...,...,...,...,...,...,...,...,...,...,...
639,Snowbell Bigleaf,Styrax grandifolius,VA,Chesapeake,31,https://www.americanforests.org?page_id=84999,https://www.americanforests.org/wp-content/plu...,36.718371,-76.246680,1,1
640,Anise-tree Yellow,Illicium parviflorum,FL,Marion,30,https://www.americanforests.org?page_id=84975,https://www.americanforests.org/wp-content/plu...,29.218245,-82.062695,1,1
641,Hawthorn Reverchon,Crataegus reverchonii,TX,Dallas,25,https://www.americanforests.org?page_id=55561,https://www.americanforests.org/wp-content/plu...,32.762040,-96.779007,1,1
642,Bayberry Evergreen,Morella caroliniensis,VA,Newport News,24,https://www.americanforests.org?page_id=56398,https://www.americanforests.org/wp-content/plu...,37.091443,-76.521389,1,1


In [174]:
def format_popup(tree):
    link, name, sci_name, points, img = tree['link'], tree['name'], tree['sci_name'], tree['points'], tree['img']
    return f'<b><a href = "{link}" target = "_blank"> {name} </a></b><br><i>{sci_name}</i><br>Points: {points} <img src = {img}></img>'

In [175]:
m = folium.Map(location=[35, -85], zoom_start = 5, tiles = 'Stamen Terrain')

states = set(trees['state'])
state_clusters = {state: folium.plugins.MarkerCluster(name = state) for state in states}

for cluster in state_clusters.values():
    cluster.add_to(m)
    
for index, tree in trees.iterrows():
    try:
        folium.Marker(
            location = [tree['lat'], tree['long']], 
            popup = format_popup(tree),
            icon = folium.map.Icon(icon = 'tree', prefix = 'fa')
        ).add_to(state_clusters[tree['state']])
    except:
        print(tree)


name                                        Soapberry Lonomea
sci_name                                   Sapindus oahuensis
state                                                      HI
county                                                Oâ€˜ahu
points                                                    123
link            https://www.americanforests.org?page_id=82099
img         https://www.americanforests.org/wp-content/plu...
lat                                                       NaN
long                                                      NaN
pctile                                                      2
qtile                                                       2
Name: 432, dtype: object
name                                    Esenbeckia Berlandier
sci_name                               Esenbeckia berlandieri
state                                                      TX
county                                            Brownsville
points                                       

In [176]:
folium.LayerControl().add_to(m)
folium.plugins.Search(state_clusters['WA'])
m

{'AdamsPA',
 'AikenSC',
 'AlachuaFL',
 'AlamedaCA',
 'AlbemarleVA',
 'AlexandriaVA',
 'AlleganyMD',
 'AlleghanyVA',
 'AmeliaVA',
 'Anne ArundelMD',
 'ApacheAZ',
 'ArlingtonVA',
 'AshlandOH',
 'AshleyAR',
 'AshtabulaOH',
 'BaltimoreMD',
 'BathVA',
 'BentonOR',
 'BexarTX',
 'BlandVA',
 'BooneMO',
 'BosqueTX',
 'BotetourtVA',
 'BrevardFL',
 'BrewsterTX',
 'BrowardFL',
 'BrownsvilleTX',
 'BrunswickVA',
 'BurlingtonNJ',
 'ButlerMO',
 'CabellWV',
 'CacheUT',
 'CalaverasCA',
 'CameronTX',
 'Cape MayNJ',
 'CarolineVA',
 'CarrollMD',
 'CarrollNH',
 'CarrollVA',
 'CecilMD',
 'CharlotteVA',
 'ChathamNC',
 'ChelanWA',
 'CherokeeAL',
 'CherokeeTX',
 'ChesapeakeVA',
 'CheshireNH',
 'ChesterPA',
 'ChicoCA',
 'ClackamasOR',
 'ClallamWA',
 'ClarkNV',
 'ClarkeGA',
 'ClatsopOR',
 'CochiseAZ',
 'CoconinoAZ',
 'CoffeeGA',
 'CollierFL',
 'ColumbiaOR',
 'ComalTX',
 'Contra CostaCA',
 'CoosNH',
 'CoosOR',
 'CowlitzWA',
 'Crow WingMN',
 'CulpeperVA',
 'CumberlandME',
 'CurryOR',
 'DallasTX',
 'DaviessKY',
 'De